# Why embed networks?

Networks by themselves are interesting objects, but they're not the way we traditionally organize data in machine learning. In almost any ML algorithm - whether you're using a neural network, or a decision tree, or whether your goal is to classify datapoints or to predict a value using regression - you'll see data organized into a matrix, where the rows represent observations and the columns represent features, or variables. Each data point, then, is traditionally represented as a point in d-dimensional space: each column gets its own axis in a plot, and each row is a single point.

## High Dimensionality of Network Data

## ML Methods for Euclidean Data

## Adjacency Spectral Embedding

## Laplacian Spectral Embedding

- calculation of similarity between two objects (e.g., cosine or euclidean distance)